## Introduction à la quantization 

Laurent cetinsoy

Les réseaux de neurones prennent beaucoup de place et il peut être difficile de les faire rentrer sur certains dispositifs embarqués. 

Il existe plusieurs méthodes pour réduire la taille et augmenter la vitesse d'executer des réseaux de neurone. Par exemple il y a ce qu'on appelle la quantization et le pruning.

Dans ce notebook on va faire une introduction à la quantization avec la librairie tensorflow lite.


## Quantization post training

Dans un premier temps on va quantifier notre réseau après l'avoir entraîné normalement. 


Entraîner un réseau de neurone convolutionnel simple avec keras pour faire de la classification MNIST (ou un autre dataset simple de votre choix si (vous en avez marre de ce dataset - https://keras.io/api/datasets/)




In [ ]:
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(60000, 28, 28, 1) / 255
x_train.shape
x_test = x_test.reshape(-1, 28, 28, 1) / 255
x_train.shape

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(200, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, validation_data=(x_test, y_test))

1875/1875 [==============================] - 131s 68ms/step - loss: 0.1034 - accuracy: 0.9683 - val_loss: 0.0380 - val_accuracy: 0.9885


Afficher le nombre de paramètre du modèle

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 200)               1843400   
                                                                 
 dense_1 (Dense)             (None, 10)                2010      
                                                        

Sauvegarder votre modèle et afficher la taille du fichier. Si on applique une bête règle de trois, quelle est la taille occupée par paramètre ? 

In [ ]:
model.save("model.h5")

import os

print("Taille du fichier: ",os.path.getsize("model.h5"), "octets" )

print("Taille occupée par parametre: ", os.path.getsize("model.h5")/model.count_params() , "octets" )

Taille du fichier:  22414000 octets
Taille occupée par parametre:  12.023220360621513 octets


On va maintenant convertir notre modèle keras en modèle tensorflow lite. 

Installer la librairie tensorflow lite créer une instance de la class TFLiteConverter à partir de votre modèle keras


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)

Convertir votre modèle et le sauvegarder dans un fichier nommé model.tflite. Sa taille est-elle plus petite ? 

In [ ]:
tflite_model = converter.convert()

with open('model_quantized.tflite', 'wb') as f:
  f.write(tflite_model)

import os

print("Taille du fichier Keras: ",os.path.getsize("model.h5"), "octets" )
print("Taille du fichier Tensorflow Lite: ",os.path.getsize("model_quantized.tflite"), "octets" )

Taille du fichier Keras:  22414000 octets
Taille du fichier Tensorflow Lite:  7460112 octets


On va maintenant spécifier des optimisations au converter. 

1. Recréer un converter

2. modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT

3. Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("model.h5")

converter_optimized = tf.lite.TFLiteConverter.from_keras_model(model)

converter_optimized.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_optimized = converter_optimized.convert()

with open('model_quantized_optimized.tflite', 'wb') as f:
    f.write(tflite_model_optimized)
  
import os
print("Taille du modèle optimisé:", os.path.getsize('model_quantized_optimized.tflite'), "octets")

Taille du modèle optimisé: 1870128 octets


Quelle type  de quantization Optimize.Default, utilise-t-elle ?


L'optimisation tf.lite.Optimize.DEFAULT utilise une combinaison de quantification de poids et d'activation ainsi que d'autres optimisations telles que l'élagage et la fusion de couches. Cette combinaison d'optimisations peut réduire considérablement la taille du modèle et améliorer les performances d'inférence sur les appareils embarqués.

Plus précisément, lors de l'utilisation de tf.lite.Optimize.DEFAULT, le convertisseur TensorFlow Lite utilise par défaut la pondération 8 bits et active la quantification. Cela signifie que les pondérations et les activations du modèle sont converties en entiers 8 bits au lieu de flottants 32 bits, ce qui réduit considérablement la taille du modèle. Le convertisseur peut également nettoyer en supprimant les poids qui ont peu d'importance relative. Enfin, le convertisseur peut fusionner certaines couches du modèle pour réduire davantage la taille du modèle. 

## Quantization aware training 

Dans cette section on va s'intéresser à l'entraînement sensible à la quantification. L'idée est de simuler les effets de la quantification pendant l'entraînement pour que le modèle ajuste les poids afin de tenir ocmpte de la quantification. 

Reprendre le modèle entraîné sur MNIST


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model("model.h5")

A l'aide de la fonction quantize de tensorflow_model_optimization, créer une seconde version de votre modèle entraîné nommé qat_model

In [ ]:
!pip install tensorflow-model-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 19.5 MB/s eta 0:00:00


In [ ]:
import tensorflow_model_optimization as tfmot

qat_model = tfmot.quantization.keras.quantize_model(model)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Compiler le modèle

In [ ]:
qat_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Afficher le summury du modèle. D'après vous ce modèle est-il quantifié ? 

In [ ]:
qat_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 28, 28, 1)        3         
 er)                                                             
                                                                 
 quant_conv2d (QuantizeWrapp  (None, 26, 26, 32)       387       
 erV2)                                                           
                                                                 
 quant_conv2d_1 (QuantizeWra  (None, 24, 24, 64)       18627     
 pperV2)                                                         
                                                                 
 quant_max_pooling2d (Quanti  (None, 12, 12, 64)       1         
 zeWrapperV2)                                                    
                                                                 
 quant_flatten (QuantizeWrap  (None, 9216)             1

Réentraîner votre modèle sur un sous ensemble des modèles sur une ou deux epochs et afficher la performance sur le train et test set

In [ ]:
qat_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

loss, accuracy = qat_model.evaluate(x_test, y_test)

print("Test set accuracy: {:.2f}%".format(accuracy * 100))

loss, accuracy = qat_model.evaluate(x_train, y_train)

print("Training set accuracy: {:.2f}%".format(accuracy * 100))

Epoch 1/2
1875/1875 [==============================] - 144s 76ms/step - loss: 0.0381 - accuracy: 0.9879 - val_loss: 0.0393 - val_accuracy: 0.9878
Epoch 2/2
313/313 [==============================] - 5s 17ms/step - loss: 0.0374 - accuracy: 0.9893
Test set accuracy: 98.93%
1875/1875 [==============================] - 34s 18ms/step - loss: 0.0121 - accuracy: 0.9956
Training set accuracy: 99.56%


Convertir votre modèle avec TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_qat_model_optimized = converter.convert()

with open('qat_model_quantized.h', 'wb') as f:
  f.write(tflite_qat_model_optimized)

with open('qat_model_quantized.tflite', 'wb') as f:
  f.write(tflite_qat_model_optimized)

/usr/local/lib/python3.9/dist-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Comparer la performance du modèle Quantified aware training, au modèle original et au modèle quantifié post training

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

# Effectuer des prédictions
tflite_qat_model_optimized.allocate_tensors()
input_details = tflite_qat_model_optimized.get_input_details()
output_details = tflite_qat_model_optimized.get_output_details()
y_pred = []
for i in range(len(x_test)):
    input_data = np.expand_dims(x_test[i], axis=0).astype(input_details[0]['dtype'])
    tflite_qat_model_optimized.set_tensor(input_details[0]['index'], input_data)
    tflite_qat_model_optimized.invoke()
    output_data = tflite_qat_model_optimized.get_tensor(output_details[0]['index'])
    y_pred.append(np.argmax(output_data))

print(y_pred)
print(y_test)
# Calculer la précision
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

[7, 2, 1, 0, 4, 1, 4, 9, 6, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6, 6, 5, 4, 0, 7, 4, 0, 1, 3, 1, 3, 4, 7, 2, 7, 1, 2, 1, 1, 7, 4, 2, 3, 5, 1, 2, 4, 4, 6, 3, 5, 5, 6, 0, 4, 1, 9, 5, 7, 8, 9, 3, 7, 4, 6, 4, 3, 0, 7, 0, 2, 9, 1, 7, 3, 2, 9, 7, 7, 6, 2, 7, 8, 4, 7, 3, 6, 1, 3, 6, 9, 3, 1, 4, 1, 7, 6, 9, 6, 0, 5, 4, 9, 9, 2, 1, 9, 4, 8, 7, 3, 9, 7, 4, 4, 4, 9, 2, 5, 4, 7, 6, 7, 9, 0, 5, 8, 5, 6, 6, 5, 7, 8, 1, 0, 1, 6, 4, 6, 7, 3, 1, 7, 1, 8, 2, 0, 2, 9, 9, 5, 5, 1, 5, 6, 0, 3, 4, 4, 6, 5, 4, 6, 5, 4, 5, 1, 4, 4, 7, 2, 3, 2, 7, 1, 8, 1, 8, 1, 8, 5, 0, 8, 9, 2, 5, 0, 1, 1, 1, 0, 9, 0, 3, 1, 6, 4, 2, 3, 6, 1, 1, 1, 3, 9, 5, 2, 9, 4, 5, 9, 3, 9, 0, 3, 6, 5, 5, 7, 2, 2, 7, 1, 2, 8, 4, 1, 7, 3, 3, 8, 8, 7, 9, 2, 2, 4, 1, 5, 9, 8, 7, 2, 3, 0, 4, 4, 2, 4, 1, 9, 5, 7, 7, 2, 8, 2, 6, 8, 5, 7, 7, 9, 1, 8, 1, 8, 0, 3, 0, 1, 9, 9, 4, 1, 8, 2, 1, 2, 9, 7, 5, 9, 2, 6, 4, 1, 5, 8, 2, 9, 2, 0, 4, 0, 0, 2, 8, 4, 7, 1, 2, 4, 0, 2, 7, 4, 3, 3, 0, 0, 3, 1, 9, 6, 5, 2, 5, 9, 7, 9, 3, 0, 4, 2, 0, 7, 1, 1, 2, 1, 

ValueError: ignored

In [ ]:
# Evaluation du modèle Quantified aware training
quantized_aware_train_loss, quantized_aware_train_accuracy = tflite_qat_model_optimized.evaluate(x_test, y_test)

# Evaluation du modèle original
original_loss, original_accuracy = model.evaluate(x_test, y_test)

# Evaluation du modèle Post training
post_training_loss, post_training_accuracy = tflite_model_optimized.evaluate(x_test, y_test)

print("Performance du modèle Quantified aware training : loss = {:.4f}, accuracy = {:.4f}".format(quantized_aware_train_loss, quantized_aware_train_accuracy))
print("Performance du modèle original : loss = {:.4f}, accuracy = {:.4f}".format(original_loss, original_accuracy))
print("Performance du modèle Post training : loss = {:.4f}, accuracy = {:.4f}".format(post_training_loss, post_training_accuracy))

AttributeError: ignored

Sauvegarder le modèle QAT et comparer les tailles des modèles

In [ ]:
qat_model.save("qat_model.h5")

import os

print("Taille du modèle original : ", os.path.getsize('model.h5'), "octets")
print("Taille du modèle QAT : ", os.path.getsize('qat_model.h5'), "octets")


Taille du modèle original :  22414000 octets
Taille du modèle QAT :  22432728 octets


Bonus : déployer votre modèle sur votre téléphone ou un dispositif embarqué si vous en disposez d'un. 

Bonus : Obtenir un modèle qui sera à la fois quantifié et élagué (prunned) en s'aidant de la documentation (https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras)

A l'aide de tensorflow lite / tensorflow lite micro 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d51e245-899d-41d6-b23b-cf3e4bbbc6ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>